## Spiders详解
　　参考：[scrapy 源码解析 （一）：启动流程源码分析(一)命令行启动](https://www.cnblogs.com/qiu-hua/p/12930422.html)<BR>
    
### 一、概述  
　　先不谈使用了各种框架的复杂情况，如scrapyd服务、redis分布式队列等。只看最简单的情况，假设只写了一个简单爬虫myspider。<BR>
　　输入命令,运行myspider爬虫：<BR>
>scrapy crawl myspider
    
　　那么，这个过程到底发生了什么？ scrapy命令与crawl、myspider参数从何而来？命令是如何执行的呢？<BR>
### 二、Scrapy命令
#### 2.1、命令里的scrapy是一个可执行文件，后面的crawl myspider是scrapy的参数。<BR>
　　可执行文件scrapy在/usr/local/python/bin目录里（linux系统使用which scrapy查看），是一个python脚本，有效代码为：<BR>
>from scrapy.cmdline import execute<BR>
<BR>
if __name__ == '__main__':<BR>
　　sys.argv[0] = re.sub(r'(-script\.pyw?|\.exe)?$', '', sys.argv\[0])<BR>
　　sys.exit(execute())<BR>

　　这个文件的作用就是从命令行里读取命令，然后传递给scrapy.cmdline包的execute()方法进行下一步操作（scrapy项目启动入口点）。<BR>
#### 2.2 scrapy命令的生成原理
　　安装scrapy后, scrapy的setup.py文件生明并指定了程序的入口点为：scrapy.cmdline.execute方法。  
### 二、execute方法
从python的第三方库目录里找到scrapy/cmdline.py文件，可以看到代码中有execute()方法：<BR>
>def execute(argv=None, settings=None):<BR>
　　if argv is None:<BR>
　　　　argv = sys.argv<BR>
    <BR>        
　　if settings is None:<BR>
　　　　settings = get_project_settings()<BR>
　　　　# set EDITOR from environment if available<BR>
　　　　try:<BR>
　　　　　　editor = os.environ['EDITOR']<BR>
　　　　except KeyError: pass<BR>
　　　　else:<BR>
　　　　　　settings['EDITOR'] = editor<BR>
　　check_deprecated_settings(settings)<BR>
  <BR>
　　inproject = inside_project()<BR>
　　cmds = _get_commands_dict(settings, inproject)<BR>
　　cmdname = _pop_command_name(argv)<BR>
　　parser = optparse.OptionParser(formatter=optparse.TitledHelpFormatter(), \<BR>
　　　　conflict_handler='resolve')<BR>
　　if not cmdname:<BR>
　　　　_print_commands(settings, inproject)<BR>
　　　　sys.exit(0)<BR>
　　elif cmdname not in cmds:<BR>
　　　　_print_unknown_command(settings, cmdname, inproject)<BR>
　　　　sys.exit(2)<BR>
    <BR>
　　cmd = cmds[cmdname]<BR>
　　parser.usage = "scrapy %s %s" % (cmdname, cmd.syntax())<BR>
　　parser.description = cmd.long_desc()<BR>
　　settings.setdict(cmd.default_settings, priority='command')<BR>
　　cmd.settings = settings<BR>
　　cmd.add_options(parser)<BR>
　　opts, args = parser.parse_args(args=argv[1:])<BR>
　　_run_print_help(parser, cmd.process_options, args, opts)<BR>
  <BR>
　　cmd.crawler_process = CrawlerProcess(settings)<BR>
　　_run_print_help(parser, _run_command, cmd, args, opts)<BR>
　　sys.exit(cmd.exitcode)<BR>
  <BR>    
def _run_command(cmd, args, opts):<BR>
　　if opts.profile:<BR>
　　　　_run_command_profiled(cmd, args, opts)<BR>
　　else:<BR>
　　　　cmd.run(args, opts)<BR>

**execute方法中的重要操作有：**<BR>
　　1.执行get_project_settings()方法，导入全局配置文件scrapy.cfg，进而导入项目的settings .py。这里用到了scrapy/utils/project.py中的get_project_settings()方法，这里先不展开。<BR>
　　2.解析命令行scrapy之后的命令，并把命令中的配置项导入到settings中，中间的一大段代码就是执行此功能，每个有效的命令均对应scrapy/commands中以命令命名的文件中的Command类。<BR>
　　3.cmd.crawler_process = CrawlerProcess(settings)这句代码建立CrawlerProcess对象，并把所有的设置settings传递给此对象。<BR>
　　4._run_print_help(parser, _run_command, cmd, args, opts)调用_run_command执行命令。如果指定了profile命令行参数，则用cProfile运行命令，cProfile是一个标准模块，这里不必考虑。无论如何，最后都会执行命令的run()方法，也就是scrapy/commands/crawl.py中Command类的run()方法。<BR>

### 三、执行crawl命令
　　scrapy/commands/crawl.py#Command:
>def run(self, args, opts):<BR>
　　if len(args) < 1:<BR>
　　　　raise UsageError()<BR>
　　elif len(args) > 1:<BR>
　　　　raise UsageError("running 'scrapy crawl' with more than one spider is no longer supported")<BR>
　　spname = args[0]<BR>
<BR>
　　self.crawler_process.crawl(spname, **opts.spargs)<BR>
　　self.crawler_process.start()<BR>

**这里有2个重要操作：**
　　1.调用CrawlerProcess的crawl方法，执行初始化（创建新爬虫对象）。<BR>
　　2.调用CrawlerProcess的start方法，正式运行。<BR>
CrawlerProcess便是scrapy运行过程中最根本的进程，是所有爬虫运行的基础。<BR>

### 四、导入配置文件
　　get_project_settings()导入配置文件的过程<BR>
　　scrapy/utils/project.py:<BR>
>...<BR>
ENVVAR = 'SCRAPY_SETTINGS_MODULE'<BR>
...<BR>
def get_project_settings():<BR>
　　if ENVVAR not in os.environ:<BR>
　　　　project = os.environ.get('SCRAPY_PROJECT', 'default')<BR>
　　　　init_env(project)<BR>
<BR>
　　settings = Settings()<BR>
　　settings_module_path = os.environ.get(ENVVAR)<BR>
　　if settings_module_path:<BR>
　　　　settings.setmodule(settings_module_path, priority='project')<BR>
<BR>
　　# XXX: remove this hack<BR>
　　pickled_settings = os.environ.get("SCRAPY_PICKLED_SETTINGS_TO_OVERRIDE")<BR>
　　if pickled_settings:<BR>
　　　　settings.setdict(pickle.loads(pickled_settings), priority='project')<BR>
<BR>
　　# XXX: deprecate and remove this functionality<BR>
　　env_overrides = {k[7:]: v for k, v in os.environ.items() if<BR>
　　　　　　　　　　　k.startswith('SCRAPY_')}<BR>
　　if env_overrides:<BR>
　　　　settings.setdict(env_overrides, priority='project')<BR>

　　return settings<BR>

　　get_project_settings会首先判断是否设置了SCRAPY_SETTINGS_MODULE环境变量，这个环境变量用来指定工程的配置模块。如果没有这个环境变量，则会调用init_env来初始化环境变量，由于我们没有设置SCRAPY_PROJECT，所以会用default默认值来执行init_env(在scrapy/utils/conf.py)。<BR>
  
### 五、init_env初始化环境变量

　　scrapy/utils/conf.py:<BR>
>def init_env(project='default', set_syspath=True):<BR>
　　"""Initialize environment to use command-line tool from inside a project<BR>
　　dir. This sets the Scrapy settings module and modifies the Python path to<BR>
　　be able to locate the project module.<BR>
　　"""<BR>
　　cfg = get_config()<BR>
　　if cfg.has_option('settings', project):<BR>
　　　　os.environ['SCRAPY_SETTINGS_MODULE'] = cfg.get('settings', project)<BR>
　　closest = closest_scrapy_cfg()<BR>
　　if closest:<BR>
　　　　projdir = os.path.dirname(closest)<BR>
　　　　if set_syspath and projdir not in sys.path:<BR>
　　　　　　sys.path.append(projdir)<BR>
<BR>
def get_config(use_closest=True):<BR>
　　"""Get Scrapy config file as a SafeConfigParser"""<BR>
　　sources = get_sources(use_closest)<BR>
　　cfg = SafeConfigParser()<BR>
　　cfg.read(sources)<BR>
　　return cfg<BR>
<BR>
def get_sources(use_closest=True):<BR>
　　xdg_config_home = os.environ.get('XDG_CONFIG_HOME') or \\<BR>
　　　　os.path.expanduser('\~/.config')<BR>
　　sources = ['/etc/scrapy.cfg', r'c:\scrapy\scrapy.cfg',<BR>
　　　　　　　　xdg_config_home + '/scrapy.cfg',<BR>
　　　　　　　　os.path.expanduser('~/.scrapy.cfg')]<BR>
　　if use_closest:<BR>
　　　　sources.append(closest_scrapy_cfg())<BR>
　　return sources<BR>

init_env首先调用get_config()获取cfg配置文件，这个配置文件获取的优先级是：<BR>
　　1./etc/scrapy.cfg，c:\scrapy\scrapy.cfg<BR>
　　2.XDG_CONFIG_HOME环境变量指定的目录下的scrapy.cfg<BR>
　　3.~/.scrapy.cfg<BR>
　　4.当前执行目录下的scrapy.cfg或者父目录中的scrapy.cfg<BR>
由于1，2，3默认都不设置，所以就使用当前执行命令下的scrapy.cfg，也就是工程目录下的scrapy.cfg。这个文件的内容很简单：<BR>
>[settings]<BR>
default = myspider.settings<BR>
[deploy]<BR>
#url = http://localhost:6800/<BR>
project = myspider<BR>

根据default = myspider.settings找到对应的配置模块，后面会执行一系列导入settings.py配置项的操作，过程其实很复杂，这里不再详细说了。
  